In [28]:
import pandas as pd
import openpyxl
import json
import os
import re
from pandas import ExcelWriter
import xlwt


In [29]:
def findKeys(lang):
    with open('./../../../crowdsource-ui/locales/{lang}.json'.format(lang=lang)) as f:
        js = json.load(f)
    
    values = []
    for key, value in js.items():
        values.append(value)
    return values

In [30]:
def extract_tags(text):
    allowed_replacements = ["x", "y", "z", "u","v","w", "a-tag-replacement"]
    regex = r"<(\S*?)[^>]*>.*?<\/\1>|<.*?\/>"
    out_txt = text
    matches = re.finditer(regex, out_txt, re.MULTILINE)
    index = 0
    matched_tags_dict = {}
    for match in matches:
        matched_tag = match.group()
        if "<b>" in matched_tag:
            continue
        if "<a" in matched_tag:
            attrs = matched_tag[matched_tag.find('<a')+2: matched_tag.find('>')]
            matched_tag_replacement = matched_tag.replace(attrs,"")
            out_txt = out_txt.replace(matched_tag, matched_tag_replacement)
            matched_tags_dict['a-tag-replacement'] = attrs
        else:
            replacement = allowed_replacements[index]
            out_txt = out_txt.replace(matched_tag, '<{}>'.format(replacement))
            index+=1
            matched_tags_dict[replacement] = matched_tag
    return out_txt , matched_tags_dict

In [44]:
def insert_tags(text, tags):
    allowed_replacements = ["x", "y", "z", "u","v","w", "a-tag-replacement"]
    regex = r"<(\S*?)[^>]*>.*?<\/\1>|<.*?\/>"
    out_txt = text
    matches = re.finditer(regex, out_txt, re.MULTILINE)
    for match in matches:
        matched_tag = match.group()
        if "<b>" in matched_tag:
            continue
        if "<a" in matched_tag:
            attrs = matched_tag[matched_tag.find('<a')+2: matched_tag.find('>')]
            matched_tag_replacement = matched_tag.replace(attrs,"")
            out_txt = out_txt.replace(matched_tag, matched_tag_replacement)
        else:
            for key, value in tags.items():
                if matched_tag == value:
                    out_txt = out_txt.replace(matched_tag, '<{}>'.format(key))
    return out_txt
    

In [31]:
def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for lang_code, lang_df in list_dfs.items():
            lang_df.to_excel(writer,lang_code)
        writer.save()
    print('done')

In [33]:
def get_dict_for_data(key, text, language, translation, extracted_tags):
    out_dict = {}
    out_dict["Actual key"] = [key]
    out_dict["English copy"] = [text]
    out_dict[language] = [translation]
    
    return out_dict

In [56]:
def create_all_trans():
    languages = [('hi', "Hindi"),('gu', "Gujarati"),('as', "Assamese"),('bn','Bengali'),('ta',"Tamil"),
                 ('te',"Telugu"),('mr',"Marathi"),('pa',"Punjabi"),('ml',"Malayalam"),('or',"Odia"),('kn',"Kannada")]
    eng_locale = 'en'
    english = 'English'
    all_dfs = {}
    for lang_code, lang_name in languages:
        lang_df = pd.DataFrame([], columns=[])
        with open('./../../../crowdsource-ui/locales/en.json') as f:
            eng_data = json.load(f)

        with open('./../../../crowdsource-ui/locales/{lang}.json'.format(lang=lang_code)) as f:
            lang_data = json.load(f)

        for key in eng_data.keys():
            processed_eng_text, extracted_tags = extract_tags(eng_data[key])

            processed_lang_text = insert_tags(lang_data[key], extracted_tags)
            print(processed_lang_text)

            data_dict = get_dict_for_data(key, processed_eng_text,lang_name, processed_lang_text, extracted_tags)

            try:
                tmp_df = pd.DataFrame.from_dict(data_dict, orient='columns')
                lang_df = lang_df.append(tmp_df, ignore_index=True)
            except Exception as e:
                print(e, "\n", data_dict, "\n\n")
    #     lang_df[lang_name] = ""
        all_dfs[lang_code] = lang_df
    return all_dfs

In [57]:
all_dfs = create_all_trans()
save_xls(all_dfs, 'all-trans.xls')

भाषा
दान
Beta
होम पेज
सुनो इंडिया
बोलो इंडिया
लिखो इंडिया
देखो इंडिया
हिंदी
(No Username)
उपयोगकर्ता बदलें
भाषा समर्थक लेवल
पहल चुनें
भाषा चुनें
बोलो इंडिया, सुनो इंडिया, लिखो इंडिया या देखो इंडिया में योगदान देकर एक भाषा समर्थक बनें। अपने योगदानों के आधार पर ब्रोंज/सिल्वर/गोल्ड/प्लैटिनम भाषा समर्थक बैज जीतें। इन्हें अपने सोशल सर्कल और अपने भाषा समुदाय में शेयर करें; इस पहल में योगदान देने के लिए अपने दोस्तों और परिवार के सदस्यों को प्रेरित करें।
भागीदारी के स्तर
इस तरह की पहल में योगदान देकर एक भाषा समर्थक बनें। भाषा समर्थक बैज जीतने के तरीके के बारे में अधिक जानकारी के लिए, कृपया पहल और अपनी पसंद की भाषा चुनें।
योगदान
सत्यापन
लेवल
बैज
लेवल और बैज अपडेट होने में 48 घंटे तक लग सकते हैं। बैज की पुष्टि करने से पहले आपके योगदान को सत्यापित किया जाएगा। मान्यता पाने का मौक़ा बनाए रखने के लिए कृपया सक्रिय रूप से योगदान करते रहें।
आपकी प्रतिक्रिया
इस प्लैटफ़ॉर्म को बेहतर बनाने के लिए हम आपकी प्रतिक्रिया जानना चाहते हैं
ईमेल
*ज़रूरी
आप किसके लिए प्रतिक्रिया देना चाहते हैं?
बोलो
सुनो
लिखो
देखो
अन्

टेक्स्ट जोड़ें
विशेष वर्णों के इस्तेमाल की अनुमति नहीं है
इनपुट फ़ील्ड भरना ज़रूरी है
कृपया अपनी चुनी हुई भाषा में टाइप करें
पूरे उत्साह के साथ रिकॉर्डिंग को लिप्यंतरित करने के लिए आपका धन्यवाद।
रद्द करें
वर्चुअल कीबोर्ड
आपकी प्रतिक्रिया सुनो इंडिया को प्रासंगिक बनाए रखने में हमारी मदद करती है, हम इस बात की सराहना करते हैं कि आपने प्रतिक्रिया देने के लिए अपना कीमती समय निकाला।
वाक्य/वाक्यों का योगदान दिया गया
आप अपना <y> भाषा समर्थक बैज जीतने से सिर्फ़ <x> वाक्य दूर हैं। <a>अधिक जानें</a>
आपने <y> वाक्यों का योगदान करके एक <x> भाषा समर्थक बैज जीता है।
अपना <y> भाषा समर्थक बैज जीतने के लिए आपका अगला लक्ष्य <x> वाक्यों तक पहुंचना है। <a>अधिक जानें</a>
वाक्य/वाक्यों का सत्यापन किया गया
आपने <y> वाक्यों को सत्यापित करके एक <x> भाषा समर्थक बैज जीता है।
अपना <y> भाषा समर्थक बैज जीतने के लिए आपका अगला लक्ष्य <x> वाक्यों तक पहुंचना है। <a>अधिक जानें</a>
क्या ऑडियो टेक्स्ट से मेल खाता है?
मूल टेक्स्ट
आपका बदलाव
सुधार करने के लिए आपका धन्यवाद!
बदलाव ज़रूरी है
भाषा की जोड़ी चुनें
इस भाषा से
बोलो इंडिया

તમારું આગામી લક્ષ્ય <y> ભાષા સમર્થક બેજ મેળવવા માટે <x> રેકોર્ડિંગ સુધી પહોંચવાનું છે. <a>વધુ જાણો</a>
વધુ માન્ય કરો
આપના સ્પીકરને ટેસ્ટ કરો
ખૂબ જ ઉત્સાહપૂર્વક રેકોર્ડિંગ્સને માન્ય કરવા બદલ આપનો આભાર.
You have helped us validate <x> recordings
ખોટુ
પ્લે કરો
ફરી પ્લે કરો
થોભાવો
ફરીથી શરૂ કરો
સુધારો
રેકોર્ડિંગમાં શું ખોટું છે તે સમજવામાં અમને મદદ કરો

તો ચાલો, આગળ વધીએ
આપે પસંદ કરેલી ભાષા
ટોચની ભાષાઓ
તેલુગુ
હિંદી
મરાઠી
તામિલ
કન્નડા
મલયાલમ
બંગાળી
ઓડિયા
અસામી
પંજાબી
ગુજરાતી
અહીં ટાઇપ કરવાનું શરૂ કરો
કોંકણી રોમન
કોંકણી ડીવી
મણીપુરી બીએન
મણીપુરી એમએમ
સંથાલી ઓએલ
સંથાલી ડીવી
બોડો
ડોંગરી
કાશ્મીરી
મૈથીલી
નેપાળી
સંસ્કૃત
સિંધી
ઉર્દૂ
કૃપા કરીને આગળ પ્રક્રિયા કરતા પહેલા <a> નિયમો અને શરતો </a> સાથે સંમત થાઓ
%language માં %hours રેકોર્ડ કર્યા
%language માં %hours માન્ય કર્યા
કલાકો
મિનિટો
સેકન્ડ્સ
5 સેકન્ડ્સ માટે રેકોર્ડિંગ
4 સેકન્ડ્સ માટે રેકોર્ડિંગ
3 સેકન્ડ્સ માટે રેકોર્ડિંગ
2 સેકન્ડ્સ માટે રેકોર્ડિંગ
1 સેકન્ડ્સ માટે રેકોર્ડિંગ
ઓડિયો ફરી ચલાવી રહ્યાં છે
ચાલુ છે
અભિનંદન!!! તમે વાક્યોની આ બેચ પૂરી કરી

আপোনাৰ ভাষাত মাত্ৰ ৫ টা বাক্য অৱদান দিয়ক
শুনা ইণ্ডিয়া
কোৱা ইণ্ডিয়া
লিখা ইণ্ডিয়া
চ‍োৱা ইণ্ডিয়া
উহুহ
পৃষ্ঠাটো বিচাৰি পোৱা নাই
দেখা গৈছে যে আপুনি ইয়াত এটা ভুল পথ লৈছে
পুণৰ আগৰ পেজলৈ ঘুৰি যাবলৈ <a>ত ক্লিক কৰক</a>
কওঁক
তাৎক্ষণিক কৌশল
অনুগ্ৰহ কৰি আপোনাৰ <b>মাইক্ৰফোনটো</b> পৰীক্ষা কৰি লওক
অনুগ্ৰহ কৰি আপোনাৰ <b>স্পীকাৰটো</b> পৰীক্ষা কৰি লওক
পিছফালৰ পৰা যাতে কোনো হুলস্থুল নহয় সেইটো নিশ্চিত কৰক
<b>ৰেকৰ্ডিং কৰাৰ আগতে </b>এবাৰ পঢ়ি লওক
ৰেকৰডিং বুটামত ক্লিক কৰি আৰম্ভ কৰক
প্ৰতিবেদন
আপোনাৰ মাইক্ৰ'ফোন আৰু স্পীকাৰ পৰীক্ষা কৰক
পৰীক্ষা
মাইক্ৰফোন পৰীক্ষা কৰক
অনুগ্ৰহ কৰি স্পষ্টকৈ কওক
পিছফালৰ কোলাহল নিম্ন/নাই
পিছৰফালে কোলাহল শুনা গৈছে
স্পীকাৰ পৰীক্ষা কৰক
<x> ছেকেণ্ডৰ পিছত ৰেকৰ্ডিং আপোনা-আপুনি বন্ধ হ'ব
অডিঅ' ৰেকৰ্ডিং অতি সৰু। অনুগ্ৰহ কৰি পুনৰ ৰেকৰ্ড কৰক
বাণীবদ্ধকৰণ আৰম্ভ কৰক
বাণীবদ্ধকৰণ বন্ধ কৰক
পুনৰ বাণীবদ্ধকৰণ কৰক
দাখিল কৰক
এৰি আগলৈ যাওঁক
আপোনাৰ কণ্ঠৰ অৱদান দিবলৈ আপোনাৰ উৎসাহৰ বাবে ধন্যবাদ।
আমাৰ <x> ভাষাত কোনো তথ্য নাই। অনুগ্ৰহ কৰি পিছত পুনৰ চেষ্টা কৰক।
ঘৰলৈ উভতি যাওক
সমল প্ৰতিবেদন কৰক
বাক্যটোত কি সমস্য

অভিযোগ
প্রশংসা
আপনার মতামত নীচে ভাগ করুন:
এখানে লিখুন..
জমা দিন
সফলভাবে জমা হয়েছে
আপনার প্রতিক্রিয়ার জন্য ধন্যবাদ!
শর্তাবলী
গোপনীয়তা নীতি
আপনার বন্ধু এবং পরিবারের সাথে ভাগ করুন
ব্যবহারকারীর বিশদ বিবরণ
সমস্ত ক্ষেত্র ঐচ্ছিক তবে আপনি জনসংখ্যা বিষয়ক বিবরণ সরবরাহ করে আপনার অবদান করা ডেটাসেটকে আরও সমৃদ্ধ করতে পারেন।
নাম
ব্যবহারকারীর নাম লিখুন
শুধুমাত্র ১২টি অক্ষর অনুমোদিত। (সংকেত: নাম.পদবী, নাম_পদবী ১২  ইত্যাদি)
দয়া করে ইমেল বা মোবাইল নম্বর ব্যবহারকারীর নাম হিসাবে ব্যবহার করবেন না
মাতৃভাষা
মাতৃভাষা নির্বাচন করুন
বয়স বিভাগ
বয়স বিভাগ নির্বাচন করুন
১০ বছর অবধি (শিশু)
১০ - ৩০ বছর (যুব)
৩০ - ৬০ বছর (প্রাপ্ত বয়স্ক)
৬০+ বছর (প্রবীণ)
লিঙ্গ
পুরুষ
মহিলা
রূপান্তরকামী - পুরুষ
রূপান্তরকামী - মহিলা
বলতে চান না
এগিয়ে চলার জন্য আপনি<a> শর্ত এবং শর্তাবলীতে</a> সম্মত হন
এগিয়ে যাওয়ার আগে যে কোনো ত্রুটি বার্তা বৈধ করুন
সম্পন্ন
ইন্ডিয়া
ড্যাশবোর্ড
একটি ভাষা নির্বাচন করুন
সকল ভাষা
শেষ ডেটা আপডেট হয়েছে
এই ভাষার জন্য আমাদের কাছে কোনো ডেটা নেই, আপনি কি এর জন্য অবদান রাখতে চান?
এখন অবদান করুন
ভাষা
বক্তারা অবদা

...

ଅବଧି ରେକର୍ଡ ହୋଇଛି
ଅବଧି ବୈଧ ହୋଇଛି
ପ୍ରଗତି ସାରଣୀ
ମାସିକ
ତ୍ରୈମାସିକ
ଲିଙ୍ଗ ବଣ୍ଟନ
ରାଜ୍ୟସ୍ତରୀୟ ବଣ୍ଟନ
ବୟସ ଗ୍ରୁପ ବଣ୍ଟନ
ଅବଦାନ ପାଇଁ ଭାଷା ଚୟନ କରନ୍ତୁ
ଭାଷାଦାନ : ଭାରତୀୟ ଭାଷା ପାଇଁ ଏକ ଭିଡ଼ ସୋର୍ସିଂ ପଦକ୍ଷେପ
I've contributed towards building open language repository for India on https://bhashini.gov.in/bhashadaan You and I can make a difference by donating our voices that can help machines learn our language and interact with us through great linguistic applications. Do your bit and empower the language?
ଆପଣଙ୍କ ଯୋଗଦାନ ଭାଷିଣୀକୁ ଏପରି ଅନେକ କାହାଣୀକୁ ଉଜ୍ଜୀବିତ କରିବା ପାଇଁ ଶକ୍ତି ପ୍ରଦାନ କରିବ ।
ସମସ୍ତଙ୍କ ଡିଜିଟାଲ ସଶକ୍ତିକରଣ ଦିଗରେ ...
ଭାଷାଦାନ ଜରିଆରେ ଭାଷାର ପ୍ରତିବନ୍ଧକ ଦୂର କରନ୍ତୁ
ଗୋଟିଏ ପ୍ଲାଟଫର୍ମ, ଏକାଧିକ କ୍ରାଉଡ ସୋର୍ସିଂ ପଦକ୍ଷେପ
ଆପଣଙ୍କ ଭାଷାକୁ ଡିଜିଟାଲ୍ ସମୃଦ୍ଧ କରିବା ପାଇଁ ତଥ୍ୟର ଏକ ଉନ୍ମୁକ୍ତ ସଂଗ୍ରହାଳୟ ସୃଷ୍ଟି କରିବାରେ ସାହାଯ୍ୟ କରନ୍ତୁ
ଅଡିଓକୁ ପାଠ୍ୟରେ ନକଲ କରି ତୁମର ଭାଷାକୁ ସମୃଦ୍ଧ କର
ଆପଣ ଅଡିଓରେ ଯାହା ଶୁଣୁଛନ୍ତି ଟାଇପ୍ କରି ଯୋଗଦାନ କରନ୍ତୁ
ଅନ୍ୟମାନଙ୍କ ଦ୍ୱାରା ଲିପିବଦ୍ଧ ହୋଇଥିବା ପାଠ୍ୟଗୁଡ଼ିକୁ ବୈଧ କରନ୍ତୁ
ଅଂଶଗ୍ରହଣ ଆରମ୍ଭ କରନ୍ତୁ
ତୁମର ସ୍ୱର ଦାନ କରି ତୁମର ଭାଷାକ

ಮಟ್ಟಗಳು
ಬ್ಯಾಜ್
ಮಟ್ಟಗಳು ಮತ್ತು ಬ್ಯಾಜ್ ಗಳು ನವೀಕರಿಸಲು 48 ಗಂಟೆಗಳವರೆಗೆ ತೆಗೆದುಕೊಳ್ಳಬಹುದು. ಬ್ಯಾಜ್ ಅನ್ನು ಧೃಡೀಕರಿಸುವ ಮೊದಲು ನಿಮ್ಮ ಕೊಡುಗೆಯನ್ನು ಮೌಲ್ಯೀಕರಿಸಲಾಗುತ್ತದೆ. ಮಾನ್ಯತೆ ಪಡೆಯುವ ಅವಕಾಶವನ್ನು ಪಡೆಯಲು ದಯವಿಟ್ಟು ಸಕ್ರಿಯವಾಗಿ ಕೊಡುಗೆ ನೀಡುತ್ತಿರಿ.
ನಿಮ್ಮ ಪ್ರತಿಕ್ರಿಯೆ
ನಾವು ಈ ವೇದಿಕೆಯನ್ನು (ಪ್ಲ್ಯಾಟ್ ಫಾರ್ಮ್ಅನ್ನು) ಸುಧಾರಿಸಲು ನಿಮ್ಮ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ಪಡೆಯಲು ಬಯಸುತ್ತೇವೆ
ಈ ಮೇಲ್
*ಅಗತ್ಯವಿದೆ
ಯಾವುದಕ್ಕಾಗಿ ಪ್ರತಿಕ್ರಿಯೆ ನೀಡುತ್ತಿರುವಿರಿ?
ಹೇಳಿ
ಕೇಳಿ
ಬರೆಯಿರಿ
ನೋಡಿ
ಇತರ
ಈ ಪುಟದ ಬಗ್ಗೆ ನಿಮ್ಮ ಅಭಿಪ್ರಾಯವೇನು
ದಯವಿಟ್ಟು ನಿಮ್ಮ ಪ್ರತಿಕ್ರಿಯೆಯ ವರ್ಗವನ್ನು ಆಯ್ಕೆಮಾಡಿ
ಐಚ್ಛಿಕ
ಒಂದು ವರ್ಗವನ್ನು ಆಯ್ಕೆಮಾಡಿ
ಸಲಹೆ
ದೋಷ
ದೂರು / ತಕರಾರು
ಮೆಚ್ಚುಗೆ
ನಿಮ್ಮ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ಈ ಕೆಳಗೆ ಹಂಚಿಕೊಳ್ಳಿ:
ಇಲ್ಲಿ ಟೈಪ್ ಮಾಡಿ ...
ಸಲ್ಲಿಸಿ
ಯಶಸ್ವಿಯಾಗಿ ಸಲ್ಲಿಸಲಾಗಿದೆ
ನಿಮ್ಮ ಪ್ರತಿಕ್ರಿಯೆಗಾಗಿ ಧನ್ಯವಾದಗಳು!
ನಿಯಮಗಳು ಮತ್ತು ಷರತ್ತುಗಳು
ಗೌಪ್ಯತಾ ನೀತಿ
ನಿಮ್ಮ ಸ್ನೇಹಿತರು ಮತ್ತು ಕುಟುಂಬದೊಂದಿಗೆ ಹಂಚಿಕೊಳ್ಳಿ
ಬಳಕೆದಾರರ ವಿವರಗಳು
ಎಲ್ಲ ಕ್ಷೇತ್ರಗಳು ಐಚ್ಛಿಕವಾಗಿವೆ ಆದರೆ ಜನಸಂಖ್ಯಾ ವಿವರಗಳನ್ನು ನೀಡುವ ಮೂಲಕ ನಿಮ್ಮ ಡೇಟಾಸೆಟ್ ಕೊಡುಗೆಯನ್ನು ನೀವು ಸಮೃದ್ಧಗೊಳಿಸಬಹುದು.
ಹೆಸರು
ಬಳಕೆದಾರ ಹೆಸರನ್ನು ನಮೂದಿಸಿ
ಕೇವಲ 12 ಅಕ್ಷರಗಳನ್ನು ಮಾತ್ರ ಅನುಮತಿಸಲಾಗಿದೆ. (ಸುಳಿವು:ಹೆಸರು.ಉಪನಾ

ಇಂಗ್ಲಿಷ್
ಹಿಂದಿ
ಅನುವಾದದ ಪ್ರಯತ್ನಗಳಿಗಾಗಿ ನಿಮ್ಮ ಧನ್ಯವಾದಗಳು
ಲಿಖೋ ಇಂಡಿಯಾವನ್ನು ಸುಸಂಗತವನ್ನಾಗಿಸಲು ನಿಮ್ಮ ಪ್ರತಿಕ್ರಿಯೆ ನಮಗೆ ಸಹಾಯ ಮಾಡುತ್ತದೆ, ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ನೀಡಲು ನೀವು ಸಮಯವನ್ನು ನೀಡಿರುವುದನ್ನು ನಾವು ಪ್ರಶಂಸಿಸುತ್ತೇವೆ.
Help your language pair achieve it’s goal of <x> translations. We know you can do more!
<y> ಚಿತ್ರಗಳನ್ನು ಮೌಲ್ಯೀಕರಿಸುವ ಮೂಲಕ ನೀವು <x> ಭಾಷಾ ಸಮರ್ಥಕ ಬ್ಯಾಜ್ ಅನ್ನು ಗಳಿಸಿದ್ದೀರಿ.
ಅನುವಾದ ಸರಿಯಾಗಿದೆಯೇ?
ಅನುವಾದಗಳನ್ನು ಮೌಲ್ಯೀಕರಿಸುವಲ್ಲಿನ ನಿಮ್ಮ ಪ್ರಯತ್ನಕ್ಕೆ ಧನ್ಯವಾದಗಳು.
ಲೇಬಲ್ ಮಾಡಲಾದ ಚಿತ್ರಗಳು
ಮೌಲ್ಯೀಕರಿಸಲಾದ ಚಿತ್ರಗಳು
ಲೇಬಲ್
ಚಿತ್ರದ ಪ್ರಕಾರ ಪಠ್ಯವನ್ನು ಟೈಪ್ ಮಾಡಿ
ಚಿತ್ರದ ಪ್ರಕಾರ ಇತರರಿಂದ ಲೇಬಲ್ ಮಾಡಲಾದ ಪಠ್ಯವನ್ನು ಮೌಲ್ಯೀಕರಿಸಿ
ನೋಡಿ ಇಂಡಿಯಾದಲ್ಲಿ ಮಾಡಲಾದ ಕೊಡುಗೆಗಳು
ಲೇಬಲ್ ಮಾಡಲಾದ ಮತ್ತು ಮೌಲ್ಯೀಕರಿಸಲಾದ ಚಿತ್ರಗಳು
ಚಿತ್ರದಲ್ಲಿನ ಪಠ್ಯವನ್ನು ಟೈಪ್ ಮಾಡಿ
ವಿಸ್ತೃತ ವೀಕ್ಷಣೆಯನ್ನು ತೆರೆಯಲು ಕ್ಲಿಕ್ ಮಾಡಿ
ಚಿತ್ರಗಳನ್ನು ಲೇಬಲ್ ಮಾಡುವಲ್ಲಿನ ನಿಮ್ಮ ಉತ್ಸಾಹಕ್ಕಾಗಿ ಧನ್ಯವಾದಗಳು.
ಲೇಬಲ್ ಮಾಡಿದ ಚಿತ್ರದಲ್ಲಿ ಏನು ತಪ್ಪಾಗಿದೆಯೆಂಬುದನ್ನು ತಿಳಿಯುವಲ್ಲಿ ನಮಗೆ ಸಹಾಯಯ ಮಾಡಿ
ದೇಖೋ ಇಂಡಿಯಾವನ್ನು ಸುಸಂಗತವನ್ನಾಗಿಸಲು ನಿಮ್ಮ ಪ್ರತಿಕ್ರಿಯೆ ನಮಗೆ ಸಹಾಯ ಮಾಡುತ್ತದೆ, ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ನೀಡಲು ನೀವು ಸಮಯವನ್ನು ನೀಡಿರುವುದನ್ನು

/Users/sreejithv/miniconda3/envs/jupyter/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  exec(code_obj, self.user_global_ns, self.user_ns)


done
